# Preparing the data

In this notebook, first processing of the data from [OMNIWeb](https://omniweb.gsfc.nasa.gov/) is done.

In [1]:
import sys
import pandas as pd
import numpy as np

%cd ..

/c/Users/u0124144/Documents/DTW_measure


The raw data is read by the following function

In [2]:
def read_raw_data(filepath='data/raw/features.csv'):
    return pd.read_csv(filepath, index_col=0)

raw = read_raw_data()

The data as if contains a year, day of year and hour column (since the low-resolution data was used). This is used to create a timestamp index for the data with the following function

In [3]:
def compose_date(years, months=1, days=1, weeks=None, hours=None, minutes=None,
                 seconds=None, milliseconds=None, microseconds=None, nanoseconds=None):
    '''Create timestamp index from raw data
    '''
    years = np.asarray(years) - 1970
    months = np.asarray(months) - 1
    days = np.asarray(days) - 1
    types = ('<M8[Y]', '<m8[M]', '<m8[D]', '<m8[W]', '<m8[h]',
             '<m8[m]', '<m8[s]', '<m8[ms]', '<m8[us]', '<m8[ns]')
    vals = (years, months, days, weeks, hours, minutes, seconds,
            milliseconds, microseconds, nanoseconds)
    return sum(np.asarray(v, dtype=t) for t, v in zip(types, vals)
               if v is not None)

Finally, some new features are added that are computed from the already available features.

In [4]:
def add_features(data):
     data['Delta_Dst'] = data.Dst.diff()
     return data

All of this is combined into one single function that returns us the first processed data.

In [5]:
def preprocess(data):
    '''Preprocess data
    '''
    data.index = compose_date(data['YEAR'], days=data['DOY'], hours=data['Hour'])
    data = data.drop(columns=['YEAR', 'DOY', 'Hour'])
    data = data.drop(columns=['pc', 'Pflux1', 'Pflux2', 'Pflux4'])
    data = add_features(data)
    return data


processed = preprocess(raw)
processed.head()

,B_scl,|B|,B_Lat,B_Lng,Bx,By_GSE,Bz_GSE,By_GSM,Bz_GSM,|RMS|,...,Pflux30,Pflux60,Flux_F,ap,f10.7,AL,AU,Magn_M,Lyman_alpha,Delta_Dst
1981-01-01 00:00:00,8.1,7.6,35.5,347.8,6.1,-1.3,4.4,-2.5,3.9,0.1,...,NaN,NaN,0,7,189.7,-14,29,2.8,0.009516,NaN
1981-01-01 01:00:00,8.7,8.6,-28.7,319.8,5.8,-4.9,-4.1,-3.7,-5.2,0.2,...,NaN,NaN,0,7,189.7,-17,13,2.4,0.009516,2.0
1981-01-01 02:00:00,8.7,8.5,-41.1,314.4,4.5,-4.6,-5.6,-3.3,-6.4,0.2,...,NaN,NaN,0,7,189.7,-44,48,2.8,0.009516,2.0
1981-01-01 03:00:00,8.4,7.8,-31.8,308.0,4.1,-5.2,-4.1,-4.5,-4.9,0.2,...,NaN,NaN,0,27,189.7,-199,112,3.1,0.009516,-2.0
1981-01-01 04:00:00,8.2,7.9,-5.6,299.0,3.8,-6.9,-0.8,-6.8,-1.5,0.2,...,NaN,NaN,0,27,189.7,-29,139,NaN,0.009516,5.0


In [6]:
processed.to_hdf('data/interim/data.h5', 'data')

NaNn values are not yet removed, and will be done when the data is prepared for input into the LSTM. This is to ensure all the values are continuous in time.

# Prepare data for input

From the data the features needed for input are extracted and transformed in the correct input format

In [36]:
# Features for the experiment
features = ['Dst', '|B|', 'Bz_GSM', 'SWDens', 'SWSpeed']
output = 'Dst'
startdate = '14-01-2001'
enddate = '01-01-2016'
time_back = 6
time_forward = 6


data = pd.read_hdf('data/interim/data.h5', 'data')
data = data[startdate:enddate]

File data/interim/data2.h5 does not exist


## Extract training and test data

First the data is split in training, validation and test set. The test set is explicitly set to the month July and December of each year. The validation set are randomly chosen months from the remaining data.

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
def extract_from_split(split, train_ind, test_ind):
    merged = None
    tr_ind = split.copy()
    te_ind = split.copy()
    train_ind.sort()
    test_ind.sort()
    for ind in test_ind[::-1]:
        del tr_ind[ind]
    for ind in train_ind[::-1]:
        del te_ind[ind]
    return pd.concat(tr_ind).sort_index(), pd.concat(te_ind).sort_index()

def controlled_train_test_split(data):
    '''Input:
        data: panda dataframe with dates
       Output:
        test set: The month July and December of each year
        train set: The remaining months
    '''
    split = [g for n,g in data.groupby(pd.Grouper(freq='M')) if g.shape[0] != 0]

    test_ind = np.hstack([[3+i*12, 7+i*12, 11+i*12] for i in range(int(len(split)/12))])
    train_ind = list(filter(lambda x: x not in test_ind, np.arange(len(split))))

    train, test = extract_from_split(split, train_ind, test_ind)

    return train, test

def split_data(data, test_size, freq='M'):
    '''Input:
        input_, output_: numpy matrices that need to be randomly split on the first index
        test_size, train_size, valid_size: percentages that sum to 1
    '''
    split = [g for n,g in data.groupby(pd.Grouper(freq=freq)) if g.shape[0] != 0]
    random = 10321
    train_ind, test_ind = train_test_split(np.arange(len(split)), random_state=random, test_size=test_size)
    train, test = extract_from_split(split, train_ind, test_ind)

    return train, test

def extract_data(data, features, output):
    if type(features) is not list:
        features = [features]
    if type(output) is not list:
        output = [output]

    data_in = data[features].copy()
    data_out = data[output].shift(-1).copy()
    return data_in, data_out

In [10]:
train, test = controlled_train_test_split(data)
train, valid = split_data(train, 0.2, 'M')

In [11]:
train_in, train_out = extract_data(train, features, output)
valid_in, valid_out = extract_data(valid, features, output)
test_in, test_out = extract_data(test, features, output)

### Normalization
The training, test and validation set has to be normalized first

In [12]:
from sklearn.preprocessing import StandardScaler

def shift_and_normalize(data, scaler=None):
    errors = data[data.isna().any(axis=1)]
    clean = data.dropna(axis=0)
    if scaler is None:
        scaler = StandardScaler()
        scaler.fit(clean.values)
    scaled_values = scaler.transform(clean.values)
    clean = pd.DataFrame(data=scaled_values, index=clean.index, columns=clean.columns)
    return pd.concat([clean, errors]).sort_index(), scaler

def preprocess_data(data, scaler=None):
    if scaler is None:
        data, scaler = shift_and_normalize(data)
    else:
        data, _ = shift_and_normalize(data, scaler)

    return data, scaler

In [13]:
tr_in, sclr = preprocess_data(train_in)
val_in, _ = preprocess_data(valid_in, sclr)
t_in, _ = preprocess_data(test_in, sclr)

### Transfrom to suitable input
Finally, the data set is processed to be used as input in the LSTM. Here, also invalid measures are filtered from the dataset.

In [14]:
def format_to_lstm_input(input_, output_, time_back, time_forward):
    num_examples = input_.shape[0]
    size = num_examples - time_forward - time_back + 1
    num_features = input_.shape[1]
    lookup = np.zeros((size, time_forward), dtype='datetime64[s]')
    X = np.zeros((size, time_back, num_features))
    y = np.zeros((size, output_.shape[1], time_forward))
    valid_ins = input_.iloc[:,0].rolling(str(time_back)+'h').apply(lambda x: True if x.shape[0] == time_back else False, raw=True)
    dates = input_.index.values
    input_ = input_.values
    output_ = output_.values
    ind = 0
    for i, val in valid_ins.reset_index(drop=True).iteritems():
        if val != 1:
            continue
        j = i + 1
        X[ind] = input_[j-time_back:j, :]
        out = output_[i:i+time_forward, :].T                
        if out.shape[1] != time_forward:
            continue
        y[ind] = out
        lookup[ind] = dates[i:i+time_forward] # Store outputdates
        if not np.isnan(X[ind]).any():
            if not np.isnan(y[ind]).any():
                ind += 1

    return X[:ind], y[:ind], lookup[:ind]

In [15]:
train_in, train_out, _ = format_to_lstm_input(tr_in, train_out, time_back, time_forward)
valid_in, valid_out, _ = format_to_lstm_input(val_in, valid_out, time_back, time_forward)
test_in, test_out, lookup = format_to_lstm_input(t_in, test_out, time_back, time_forward)

### Importing storm data 
Now we extract the storms that occur in the testset.

In [24]:
def get_storm_dates(stormname='data/external/semi_auto_storms_8114.csv'):
    return pd.read_csv(stormname, index_col=0, dtype={'date1': str, 'date2': str},
                       parse_dates=['date1', 'date2'])

def get_storms(data, storm_dates):
    '''Check which storms lie in the given dataset
    Input:
        data: dataset with dates
        storm_dates: list of known storm occurences
    Output:
        measured features during the storm
        storm-dates that lie in data
    '''
    rs = []
    valid_storms = []
    for (_, dates) in storm_dates.iterrows():
        ss = data.loc[dates[0]:dates[1]]
        if ss.shape[0] != 0:
            rs.append(ss)
            valid_storms.append((dates[0].strftime('%Y-%m-%d'), dates[1].strftime('%Y-%m-%d')))
    print('Number of storms in dataset: {}'.format(len(valid_storms)))
    return pd.concat(rs), np.array(valid_storms)

storm_dates = get_storm_dates()
test_storms, test_storm_dates = get_storms(test, storm_dates)

Number of storms in dataset: 17


### Store the data

In [29]:
import h5py

def store_data_sets(train_in, train_out, valid_in, valid_out, test_in, test_out, lookup, test_storm_dates, fname='data/processed/datasets.h5'):
    with h5py.File(fname, 'w') as f:
        train = f.create_group('train_sets')
        train.create_dataset('train_in', data=train_in)
        train.create_dataset('train_out', data=train_out)
        valid = f.create_group('valid_sets')
        valid.create_dataset('valid_in', data=valid_in)
        valid.create_dataset('valid_out', data=valid_out)
        test = f.create_group('test_sets')
        test.create_dataset('test_in', data=test_in)
        test.create_dataset('test_out', data=test_out)
        test.create_dataset('lookup', data=lookup.astype(np.long), dtype='int64')
        storms = test.create_group('storms')
        storms.create_dataset('storm_dates', data=test_storm_dates.astype('S'))
    
store_data_sets(train_in, train_out, valid_in, valid_out, test_in, test_out, lookup, test_storm_dates)